In [1]:
# Import the libraries
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
# Define a function to compute BS formula
def put_BS(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) +
         (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    
    d2 = d1 - sigma * np.sqrt(T)

    return  K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

In [3]:
# Fix the pricing parameters
S0 = 100
T = 1
K = 101
r = 0.05
sigma = 0.3

In [4]:
# Compute the BS price of the put
P0_BS = put_BS(S0, K, T, r, sigma)
P0_BS

9.829790971709542

In [5]:
# Import NumPy's pseudo-random generator
from numpy.random import default_rng

In [6]:
# Create the generator
rng = default_rng(100)

In [7]:
# Generate the standard normal numbers
M = 1000000
epsilons = rng.standard_normal(M)

In [8]:
# Generate ST
ST = S0 * np.exp((r - sigma**2 / 2) * T +
                sigma * np.sqrt(T) * epsilons)

In [9]:
# Generate the payoff of the put
PT = np.maximum(K - ST, 0)

In [10]:
# Compute the Monte Carlo price
P0_MC = np.exp(-r * T) * PT.mean()
P0_MC

9.838236547547348

In [11]:
# Define the payoff function
def phi(x):
    return np.maximum(K - x, 0)

In [12]:
# Define the lognormal density
mu_t = np.log(S0) + (r - sigma**2 / 2) * T

sigma_t = sigma * np.sqrt(T)

from scipy.stats import lognorm

def l(x):
    return lognorm.pdf(x, s=sigma_t, scale=np.exp(mu_t))

In [13]:
# Define the integrand function
def f_int(x):
    return phi(x) * l(x)

In [14]:
# Compute the integral
from scipy.integrate import quad

I = quad(f_int, -np.infty, np.infty)[0]
I

10.333775133348153

In [15]:
# Compute the integral price
P0_int = np.exp(-r * T) * I
P0_int

9.829790973014553